# Preprocessing

In [1]:
import pandas as pd
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, TargetEncoder

CLEANED_DATA_PATH = '../data/processed/lending-club-cleaned.csv'
# Save separate train and test files to maintain the vault!
TRAIN_DATA_PATH = '../data/processed/train_fe.csv'
TEST_DATA_PATH = '../data/processed/test_fe.csv'
PIPELINE_PATH = '../models/preprocessor.joblib'

In [2]:
df = pd.read_csv(CLEANED_DATA_PATH)
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,mort_acc_missing,emp_length_missing,credit_history_years,issue_years
0,5000,36 months,11.53,164.96,B,B5,NaN,OWN,21000.0,Verified,...,55.7,27.0,w,Individual,0.0,1.0,0,1,40.668036,3.501711
1,30375,36 months,13.49,1030.64,C,C2,10+ years,OWN,108000.0,Verified,...,48.7,14.0,w,Individual,2.0,0.0,0,0,11.668720,2.499658
2,13250,36 months,18.75,484.02,D,D3,10+ years,MORTGAGE,50000.0,Source Verified,...,53.0,19.0,f,Individual,2.0,0.0,0,0,26.748802,5.582478
3,7000,36 months,7.07,216.37,A,A2,10+ years,MORTGAGE,50000.0,Not Verified,...,31.8,28.0,w,Individual,1.0,0.0,0,0,13.330595,1.415469
4,5000,36 months,7.29,155.05,A,A4,10+ years,OWN,60000.0,Not Verified,...,32.8,22.0,f,Individual,NaN,0.0,1,0,19.997262,7.997262


### Split Data

In [3]:
df = pd.read_csv(CLEANED_DATA_PATH)
df_prep = df.copy()

df_prep['loan_status'] = df_prep['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1})

X = df_prep.drop(columns=['loan_status'])
y = df_prep['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training Data Shape: {X_train.shape}")
print(f"Testing Data Shape:  {X_test.shape}")

Training Data Shape: (3995, 25)
Testing Data Shape:  (999, 25)


### Define Pipeline

In [4]:
# Scikit-learn >= 1.2 allows us to output clean Pandas DataFrames instead of messy Numpy Arrays
pd.set_option('display.max_columns', None)

# Blueprint 1: emp_length (Needs BOTH Mode Imputation AND Ordinal Encoding)
emp_categories = ['< 1 year', '1 year', '2 years', '3 years', '4 years', 
                  '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years']

emp_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[emp_categories]))
])

# Blueprint 2: Standard Ordinals (No missing values expected here)
ord_cols = ['term', 'grade', 'sub_grade', 'verification_status']
ord_categories = [
    [' 36 months', ' 60 months'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
     'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5',
     'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
     'G1', 'G2', 'G3', 'G4', 'G5'],
    ['Not Verified', 'Verified', 'Source Verified']
]

ord_pipe = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories=ord_categories))
])

# Blueprint 3: mort_acc (Needs Median Imputation)
mort_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [5]:
handled_categorical_cols = ['emp_length', 'term', 'grade', 'sub_grade', 'verification_status', 'purpose']

all_categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

ohe_cols = [col for col in all_categorical_cols if col not in handled_categorical_cols]

print(f"Dynamically detected columns for OHE: {ohe_cols}")

Dynamically detected columns for OHE: ['home_ownership', 'initial_list_status', 'application_type']


In [6]:
preprocessor = ColumnTransformer(transformers=[
    ('emp', emp_pipe, ['emp_length']),
    ('ord', ord_pipe, ord_cols),
    ('mort', mort_pipe, ['mort_acc']),
    ('target', TargetEncoder(target_type='binary'), ['purpose']),
    
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), ohe_cols)
    
], remainder='passthrough') 

preprocessor.set_output(transform="pandas")
preprocessor.fit(X_train, y_train)

,transformers,"[('emp', ...), ('ord', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None


### Apply Transformation

In [7]:
# Apply the learned rules to both sets securely
X_train_fe = preprocessor.transform(X_train)
X_test_fe = preprocessor.transform(X_test)

# Optional: Scikit-learn adds prefixes like 'remainder__' to column names. 
# This quick loop cleans them up so your columns look normal again.
X_train_fe.columns = [col.split('__')[-1] for col in X_train_fe.columns]
X_test_fe.columns = [col.split('__')[-1] for col in X_test_fe.columns]

print("Transformations applied.")
print(f"Final Training Features Shape: {X_train_fe.shape}")
print(f"Final Testing Features Shape:  {X_test_fe.shape}")

# Verify no nulls remain in the features
print(f"\nRemaining Nulls in Train:\n{X_train_fe.isnull().sum()[X_train_fe.isnull().sum() > 0]}")

Transformations applied.
Final Training Features Shape: (3995, 27)
Final Testing Features Shape:  (999, 27)

Remaining Nulls in Train:
Series([], dtype: int64)


### Save Dataset & Pipeline

In [8]:
os.makedirs('../models', exist_ok=True)

train_df = pd.concat([X_train_fe, y_train], axis=1)
test_df = pd.concat([X_test_fe, y_test], axis=1)

train_df.to_csv(TRAIN_DATA_PATH, index=False)
test_df.to_csv(TEST_DATA_PATH, index=False)

# Save the actual fitted pipeline! 
joblib.dump(preprocessor, PIPELINE_PATH)

print(f"Saved Train Data to: {TRAIN_DATA_PATH}")
print(f"Saved Test Data to:  {TEST_DATA_PATH}")
print(f"Saved Preprocessor Pipeline to: {PIPELINE_PATH}")

Saved Train Data to: ../data/processed/train_fe.csv
Saved Test Data to:  ../data/processed/test_fe.csv
Saved Preprocessor Pipeline to: ../models/preprocessor.joblib
